In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
# Set the path to the Food-101 dataset
rootfolder = '/kaggle/input/indonesian-foodv2/indonesian-food/'
sampleimgpath = '/kaggle/input/indonesian-foodv2/indonesian_food/food/'
sampleimages = ['ayam_bakar/ayam_bakar_(10).jpg', 'bakso/bakso-20004.jpg', 'gado_gado/gado gado-20001.jpg', 'rendang/Rendang-11.jpg', 'sate/sate-20003.jpg']

# configurations
epoch_count = 25
img_size = (224,224)
img_input_shape = (224,224, 3)  # can be tried with lower size to train model fast
batch_size = 32

# Set the paths for the data directory and the saved model
data_dir = '/kaggle/input/indonesian-foodv2/indonesian_food/food/'
saved_model_path = 'ModelML.h5'

In [3]:
# Limiting as of now for five kinds of foods
# Get the list of folders to use
use_folders = ['ayam_bakar', 'bakso', 'gado_gado', 'rendang', 'sate']

# Get the list of subfolders in the dataset directory
subfolders = os.listdir(data_dir)

# Create a list of the subfolders to use
subfolders_to_use = [subfolder for subfolder in subfolders if subfolder in use_folders]

# Uncomment following line if want to train the model for all 101 foods
# subfolders_to_use = subfolders

# Define the number of classes
num_classes = len(subfolders_to_use)

# Get the list of class names
class_names = sorted(subfolders_to_use)

In [4]:
# Define the image generator for training data and validation data (20%)
# Also apply multiple augmentations
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)


# Define the training and validation generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    classes=subfolders_to_use,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    classes=subfolders_to_use,
    class_mode='categorical',
    subset='validation')


Found 7979 images belonging to 5 classes.
Found 1992 images belonging to 5 classes.


In [5]:
# Load pre-trained VGG16 model without top layers
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=img_input_shape)

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
# Freeze pre-trained layers
for layer in vgg16_model.layers:
    layer.trainable = False  

# Add custom top layers
x = vgg16_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine pre-trained model with custom top layers
model = Model(inputs=vgg16_model.input, outputs=predictions)

In [7]:
# Compile the model with the Adam optimizer and categorical cross-entropy loss. can be tried with other optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Train model
model.fit(train_generator, epochs=epoch_count, validation_data=val_generator)

Epoch 1/25
250/250 [==============================] - 167s 630ms/step - loss: 0.9061 - accuracy: 0.7353 - val_loss: 0.6118 - val_accuracy: 0.7726
Epoch 2/25
250/250 [==============================] - 127s 509ms/step - loss: 0.4622 - accuracy: 0.8361 - val_loss: 0.5253 - val_accuracy: 0.8112
Epoch 3/25
250/250 [==============================] - 128s 513ms/step - loss: 0.4250 - accuracy: 0.8520 - val_loss: 0.3486 - val_accuracy: 0.8720
Epoch 4/25
250/250 [==============================] - 128s 511ms/step - loss: 0.4072 - accuracy: 0.8560 - val_loss: 0.3190 - val_accuracy: 0.8850
Epoch 5/25
250/250 [==============================] - 127s 509ms/step - loss: 0.3800 - accuracy: 0.8679 - val_loss: 0.3159 - val_accuracy: 0.8845
Epoch 6/25
250/250 [==============================] - 127s 507ms/step - loss: 0.3564 - accuracy: 0.8732 - val_loss: 0.2857 - val_accuracy: 0.8931
Epoch 7/25
250/250 [==============================] - 127s 506ms/step - loss: 0.3482 - accuracy: 0.8753 - val_loss: 0.3647 -

In [9]:
# Evaluate the model on the validation set
score = model.evaluate(val_generator, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])
print('-----------------------------------\n')

# Save the trained model
model.save(saved_model_path)

Validation loss: 0.30171942710876465
Validation accuracy: 0.8960843086242676
-----------------------------------



In [10]:
def GetFoodName(imgpath):
    # Load the image
    img = tf.keras.preprocessing.image.load_img(imgpath, target_size=img_size)

    # Convert the image to an array
    img_array = tf.keras.preprocessing.image.img_to_array(img)

    # Reshape the array to match the input shape of the model
    img_array = np.expand_dims(img_array, axis=0)

    # Normalize the image data
    img_array = img_array / 255.0

    # Make a prediction using the trained model
    prediction = model.predict(img_array)
    print(prediction)

    # Get the predicted class label
    predicted_class_idx = np.argmax(prediction, axis=-1)[0]
    predicted_class_label = class_names[predicted_class_idx]
    return predicted_class_label

In [11]:
# Reload the trained model
model = k.models.load_model('/kaggle/working/' + saved_model_path)

# Check of sample images and print the predicted class label
for img in sampleimages:
    sampleimage = sampleimgpath + img
    predicted_class_label = GetFoodName(sampleimage)
    print("Predicted class label: ", predicted_class_label)

1/1 [==============================] - 0s 414ms/step
[[2.1012358e-03 3.9015056e-04 9.8933440e-01 7.3632095e-03 8.1090134e-04]]
Predicted class label:  gado_gado
1/1 [==============================] - 0s 21ms/step
[[4.3359782e-09 9.9996614e-01 1.8078175e-10 2.6750843e-07 3.3575263e-05]]
Predicted class label:  bakso
1/1 [==============================] - 0s 17ms/step
[[2.1057196e-05 2.5117600e-01 7.8178499e-07 4.2905463e-05 7.4875927e-01]]
Predicted class label:  sate
1/1 [==============================] - 0s 17ms/step
[[1.4765030e-03 2.5891716e-06 1.2272199e-03 9.9721861e-01 7.5110860e-05]]
Predicted class label:  rendang
1/1 [==============================] - 0s 17ms/step
[[9.9912089e-01 9.7985105e-07 2.0181101e-04 6.6332682e-04 1.3011687e-05]]
Predicted class label:  ayam_bakar
